In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os

# set env flags to catch BLAS used for scipy/numpy 
# to only use 1 cpu, n_cpus will be totally controlled by csky
if False:
    os.environ['MKL_NUM_THREADS'] = "1"
    os.environ['NUMEXPR_NUM_THREADS'] = "1"
    os.environ['OMP_NUM_THREADS'] = "1"
    os.environ['OPENBLAS_NUM_THREADS'] = "1"
    os.environ['VECLIB_MAXIMUM_THREADS'] = "1"

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = 'w'
mpl.rcParams['savefig.facecolor'] = 'w'
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import csky as cy
from csky import cext
import numpy as np
import astropy
#from icecube import astro
import histlite as hl
import healpy
import healpy as hp
import socket
import pickle
from scipy import stats
import copy
healpy.disable_warnings()
plt.rc('figure', facecolor = 'w')
plt.rc('figure', dpi=100)

## Define Settings

In [ ]:
selection_version = 'version-001-p00'

host_name = socket.gethostname()

if 'cobalt' in host_name:
    print('Working on Cobalts')
    #data_prefix = '/data/user/ssclafani/data/cscd/final'
    #ana_dir = '/data/user/ssclafani/data/analyses/'
    plot_dir = cy.utils.ensure_dir('/data/user/mhuennefeld/data/analyses/DNNCascadeCodeReview/unblinding_checks/plots/unblinding/galactic_latitude_hist')
    
else:
    raise ValueError('Unknown host:', host_name)

In [ ]:
for dir_path in [plot_dir]:
    if not os.path.exists(dir_path):
        print('Creating directory:', dir_path)
        os.makedirs(dir_path)

## Load Data

In [ ]:
repo = cy.selections.Repository()
specs = cy.selections.DNNCascadeDataSpecs.DNNC_10yr

In [ ]:
%%time

ana = cy.get_analysis(
    repo, selection_version, specs, 
    #gammas=np.r_[0.1:6.01:0.125],
)

In [ ]:
a = ana.anas[0]
a.sig

In [ ]:
a.bg_data

## Helpers

In [ ]:
from cycler import cycle
from copy import deepcopy

soft_colors = cy.plotting.soft_colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


def get_bias_allt(tr, ntrials=200, n_sigs=np.r_[:101:10], quiet=False):
    trials = [
        (None if quiet else print(f'\r{n_sig:4d} ...', end='', flush=True))
        or
        tr.get_many_fits(ntrials, n_sig=n_sig, logging=False, seed=n_sig)
        for n_sig in n_sigs]
    if not quiet:
        print()
    for (n_sig, t) in zip(n_sigs, trials):
        t['ntrue'] = np.repeat(n_sig, len(t))
    allt = cy.utils.Arrays.concatenate(trials)
    return allt

def get_color_cycler():
    return cycle(colors)

def plot_ns_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)

    h = hl.hist((allt.ntrue, allt.ns), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(ax.set_ylim(lim))
    ax.plot(lim, lim, **expect_kw)
    ax.set_aspect('equal')

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$n_s$')
    ax.grid()

def plot_gamma_bias(ax, tr, allt, label=''):

    n_sigs = np.unique(allt.ntrue)
    dns = np.mean(np.diff(n_sigs))
    ns_bins = np.r_[n_sigs - 0.5*dns, n_sigs[-1] + 0.5*dns]
    expect_kw = dict(color='C0', ls='--', lw=1, zorder=-10)
    expect_gamma = tr.sig_injs[0].flux[0].gamma

    h = hl.hist((allt.ntrue, allt.gamma), bins=(ns_bins, 100))
    hl.plot1d(ax, h.contain_project(1),errorbands=True, 
              drawstyle='default', label=label)
    lim = ns_bins[[0, -1]]
    ax.set_xlim(lim)
    ax.set_ylim(1, 4)
    ax.axhline(expect_gamma, **expect_kw)

    ax.set_xlabel(r'$n_{inj}$')
    ax.set_ylabel(r'$\gamma$')
    ax.grid()

def plot_bkg_trials(
            bg, fig=None, ax=None, 
            label='{} bg trials', 
            label_fit=r'$\chi^2[{:.2f}\mathrm{{dof}},\ \eta={:.3f}]$', 
            color=colors[0],
            density=False,
            bins=50,
        ):
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    if density:
        h = bg.get_hist(bins=bins).normalize(density=True)
    else:
        h = bg.get_hist(bins=bins)
    if label is not None:
        label = label.format(bg.n_total)
    hl.plot1d(ax, h, crosses=True, color=color, label=label)

    # compare with the chi2 fit:
    if hasattr(bg, 'pdf'):
        x = h.centers[0]
        norm = h.integrate().values
        if label_fit is not None:
            label_fit = label_fit.format(bg.ndof, bg.eta)
        if density:
            ax.semilogy(x, bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)
        else:
            ax.semilogy(x, norm * bg.pdf(x), lw=1, ls='--', label=label_fit, color=color)

    ax.set_xlabel(r'TS')
    if density:
        ax.set_ylabel(r'Density')
    else:
        ax.set_ylabel(r'number of trials')
    ax.legend()
        
    return fig, ax

##### Helpers for event injection

In [ ]:
def get_flux(tr):
    for key, item in tr.llh_kw['conf'].items():
        if 'flux' == key:
            return item
        if isinstance(item, dict):
            for key_sub, item_sub in item.items():
                if key_sub == 'flux':
                    return item_sub
    return None


def get_injected_events(tr, n_sig, seed=None):
    injector = tr.sig_injs[0]
    original_keep = [k for k in injector.keep]
    if 'energy' not in injector.keep:
        injector.keep += ['energy']
    if 'true_energy' not in injector.keep:
        injector.keep += ['true_energy']
    injected_events = injector.inject(n_sig, seed=seed)[0][0]
    injector.keep = original_keep
    return injected_events
    
def get_bkg_events(tr, seed=None):
    randomizer = tr.bg_injs[0].randomizers[0]
    bg_data_cpy = cy.utils.Events(a.bg_data)
    randomizer(bg_data_cpy, seed=seed)
    return bg_data_cpy


## Setup Analysis

In [ ]:
import sys
sys.path.insert(0, '../..')

import config as cg

cg.base_dir = '/data/user/mhuennefeld/data/analyses/unblinding_v1.0.0/'

In [ ]:
def get_gp_tr(template_str, cutoff=np.inf, gamma=None, ana=ana, cpus=20):
    cutoff_GeV = cutoff * 1e3
    gp_conf = cg.get_gp_conf(
        template_str=template_str, gamma=gamma, 
        cutoff_GeV=cutoff_GeV, base_dir=cg.base_dir)
    
    if template_str == 'pi0' and cutoff != np.inf or ana.keys != ['DNNCascade_10yr']:
        print('Removing dir!', cutoff)
        gp_conf.pop('dir')

    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr

def get_template_tr(template, gamma=2.7, cutoff_tev=np.inf, cpus=20):
    cutoff_gev = cutoff_tev * 1000.
    gp_conf = {
        'template': template,
        'flux': cy.hyp.PowerLawFlux(gamma, energy_cutoff=cutoff_gev),
        'randomize': ['ra'],
        'fitter_args': dict(gamma=gamma),
        'sigsub': True,
        'update_bg': True,
        'fast_weight': False,
    }
    tr = cy.get_trial_runner(gp_conf, ana=ana, mp_cpus=cpus)
    return tr


#### Get TrialRunners

In [ ]:
tr_dict = {
    'pi0': get_gp_tr('pi0'),
    'kra5': get_gp_tr('kra5'),
    'kra50': get_gp_tr('kra50'),
}

#### Get Results for each template

In [ ]:
res_dict = {}
for key in tr_dict.keys():
    f_path = os.path.join(
        cg.base_dir, 
        'gp/results/{}/{}_unblinded.npy'.format(key, key), 
    )
    if os.path.exists(f_path):
        res_dict[key] = np.load(f_path)

In [ ]:
ns_dict = {
    'pi0': 748.11,
    'pi0_lowE': 623.67,
    'kra5': 273.24,
    'kra50': 208.95,
    'snr': 218.45,
    'pwn': 279.61,
    'unid': 237.90,
}

#### Helper functions for projection on Galactic Latitude

In [ ]:
from astropy.coordinates import SkyCoord


def ra_dec_to_lon_lat(ra, dec, unit='rad'):
    c_icrs = SkyCoord(ra=ra, dec=dec, unit=unit, frame='icrs')
    c_gal = c_icrs.galactic
    return c_gal.data.lon.rad, c_gal.data.lat.rad

 
#np.rad2deg(ra_dec_to_lon_lat(np.deg2rad([083.63308, 266.41500889]), np.deg2rad([22.01450, -29.00611111])))


In [ ]:
from csky import cext
from scipy.integrate import quad
from tqdm.notebook import tqdm_notebook as tqdm

def compute_dec_pdf(src_decs, ev_decs, ev_sigmas):
    
    src_decs = np.atleast_1d(src_decs)
    
    sigsub = True
    kent_min = 0.
    cut_pdf_threshold = -1
    cut_n_sigma = float('inf')
    ev_ras = np.zeros_like(ev_decs)
    src_ras = np.zeros_like(src_decs)
    src_exts = np.zeros_like(src_decs)
    
    sig_data, sigsub_data, row, column = cext.kent_1D_space_pdf(
        ev_decs, ev_ras, ev_sigmas, src_decs, src_ras, src_exts, 
        sigsub, cut_n_sigma, cut_pdf_threshold, kent_min,
    )
    return sigsub_data.reshape([-1, len(src_decs)]) * 2*np.pi * np.cos(src_decs)

def compute_dec_cdf(src_dec_min, src_dec_max, ev_decs, ev_sigmas, n_points=100):
    
    bins_src_decs = np.linspace(src_dec_min, src_dec_max, 100)
    src_dec_width = np.diff(bins_src_decs)[0]
    assert np.allclose(np.diff(bins_src_decs), src_dec_width)

    src_decs = bins_src_decs[:-1] + 0.5 * src_dec_width

    res = compute_dec_pdf(src_decs, ev_decs=ev_decs, ev_sigmas=ev_sigmas)
    return np.sum(res, axis=1) * src_dec_width

def compute_abs_dec_bin_contribution(ev_decs, ev_sigmas, abs_dec_bins):
    n_bins = len(abs_dec_bins) - 1
    bin_contribution = np.empty((len(ev_decs), n_bins))
    
    for i in tqdm(range(n_bins), total=n_bins):
        
        # positive dec range
        bin_contribution[:, i] = compute_dec_cdf(
            src_dec_min=abs_dec_bins[i], 
            src_dec_max=abs_dec_bins[i+1], 
            ev_decs=ev_decs, 
            ev_sigmas=ev_sigmas,
        )
        
        # negative dec range
        bin_contribution[:, i] += compute_dec_cdf(
            src_dec_min=-abs_dec_bins[i+1], 
            src_dec_max=-abs_dec_bins[i], 
            ev_decs=ev_decs, 
            ev_sigmas=ev_sigmas,
        )
    
    return bin_contribution

dec_min = -np.pi/2.
dec_max = np.pi/2.
ev_decs = [-0.1]
ev_sigmas = [0.1]
print('Discrete', compute_dec_cdf(dec_min, dec_max, ev_decs=ev_decs, ev_sigmas=ev_sigmas))
print('Quad', quad(compute_dec_pdf, dec_min, dec_max, args=(ev_decs, ev_sigmas)))
bin_contribution = compute_abs_dec_bin_contribution(ev_decs=ev_decs, ev_sigmas=ev_sigmas, abs_dec_bins=np.linspace(0, np.pi/2, 10))
print(bin_contribution, np.sum(bin_contribution))


#### Compute Bin contribution

In [ ]:
def get_bin_contributions(mask_func=None, n_reps=10, template_string='pi0', bins=np.linspace(0, 90, 15)):
    bkg_events = cy.utils.Arrays.concatenate([get_bkg_events(tr=tr_dict[template_string]) for i in range(n_reps)])
    bkg_lon, bkg_lat = ra_dec_to_lon_lat(ra=bkg_events.ra, dec=bkg_events.dec)

    inj_events = get_injected_events(tr_dict[template_string], n_sig=int(ns_dict[template_string]))
    inj_lon, inj_lat = ra_dec_to_lon_lat(ra=inj_events.ra, dec=inj_events.dec)

    exp_lon, exp_lat = ra_dec_to_lon_lat(ra=a.bg_data.ra, dec=a.bg_data.dec)
    
    if mask_func is None:
        mask_bkg = np.ones_like(bkg_lat, dtype=bool)
        mask_inj = np.ones_like(inj_lat, dtype=bool)
        mask_exp = np.ones_like(exp_lat, dtype=bool)
    else:
        mask_bkg = mask_func(bkg_events)
        mask_inj = mask_func(inj_events)
        mask_exp = mask_func(a.bg_data)
        
    bkg_events_m = bkg_events[mask_bkg]
    inj_events_m = inj_events[mask_inj]
    exp_events_m = a.bg_data[mask_exp]
        
    bkg_weights = np.ones_like(bkg_lat[mask_bkg]) / n_reps
    
    plot_bkg = np.rad2deg(np.abs(bkg_lat[mask_bkg]))
    plot_inj = np.rad2deg(np.abs(inj_lat[mask_inj]))
    plot_exp = np.rad2deg(np.abs(exp_lat[mask_exp]))
    
    bin_mids = bins[:-1] + 0.5 * np.diff(bins)
    bin_width = np.diff(bins)[0]
    assert np.allclose(np.diff(bins), bin_width)
        
    hist_bkg, _ = np.histogram(plot_bkg, bins=bins, weights=bkg_weights)
    hist_inj, _ = np.histogram(plot_inj, bins=bins)
    hist_exp, _ = np.histogram(plot_exp, bins=bins)
    
    # compute contribution to bins
    bin_prob_bkg = compute_abs_dec_bin_contribution(
        ev_decs=bkg_lat[mask_bkg], ev_sigmas=bkg_events_m.sigma, abs_dec_bins=np.deg2rad(bins))
    bin_prob_inj = compute_abs_dec_bin_contribution(
        ev_decs=inj_lat[mask_inj], ev_sigmas=inj_events_m.sigma, abs_dec_bins=np.deg2rad(bins))
    bin_prob_exp = compute_abs_dec_bin_contribution(
        ev_decs=exp_lat[mask_exp], ev_sigmas=exp_events_m.sigma, abs_dec_bins=np.deg2rad(bins))
    
    hist_prob_bkg = np.sum(bin_prob_bkg, axis=0) / float(n_reps)
    hist_prob_inj = np.sum(bin_prob_inj, axis=0)
    hist_prob_exp = np.sum(bin_prob_exp, axis=0)
    print('bin_prob_inj', bin_prob_inj.shape)
    print('bin_prob_exp', bin_prob_exp.shape)
    
    sig_fraction = np.sum(hist_inj) / np.sum(hist_bkg)
    print('sig_fraction', sig_fraction, np.sum(hist_prob_inj) / np.sum(hist_prob_bkg))
    hist_combined = hist_bkg * (1. - sig_fraction) + hist_inj
    hist_prob_combined = hist_prob_bkg * (1. - sig_fraction) + hist_prob_inj
    
    bin_contributions = {
        'bins': bins,
        'n_reps': n_reps,
        
        'bkg_events_m': bkg_events_m,
        'inj_events_m': inj_events_m,
        
        'plot_bkg': plot_bkg,
        'plot_inj': plot_inj,
        'plot_exp': plot_exp,
        'bin_prob_bkg': bin_prob_bkg,
        'bin_prob_inj': bin_prob_inj,
        'bin_prob_exp': bin_prob_exp,
        
        'hist_bkg': hist_bkg,
        'hist_inj': hist_inj,
        'hist_exp': hist_exp,
        'hist_combined': hist_combined,
        'hist_prob_bkg': hist_prob_bkg,
        'hist_prob_inj': hist_prob_inj,
        'hist_prob_exp': hist_prob_exp,
        'hist_prob_combined': hist_prob_combined,
    }
    return bin_contributions
        


In [ ]:
reload = True

bin_args = (0, 90, 30) # 15
bins = np.linspace(*bin_args)

bin_contributions_file = os.path.join(plot_dir, 'bin_contributions_{}_{}_{}.pkl'.format(*bin_args))
if os.path.exists(bin_contributions_file) and reload:
    print('Reloading data from file')
    with open(bin_contributions_file, 'rb') as handle:
        bin_contributions = pickle.load(handle)
else:
    print('Computing data')
    bin_contributions = get_bin_contributions(n_reps=10, bins=bins)    
    with open(bin_contributions_file, 'wb') as handle:
        pickle.dump(bin_contributions, handle, protocol=pickle.HIGHEST_PROTOCOL)

#### Make Projection Histogram

In [ ]:
def make_projection_hist(bin_contributions, mask_func=None):

    inj_events = bin_contributions['inj_events_m']
    inj_lon, inj_lat = ra_dec_to_lon_lat(ra=inj_events.ra, dec=inj_events.dec)

    bkg_events = bin_contributions['bkg_events_m']
    bkg_lon, bkg_lat = ra_dec_to_lon_lat(ra=bkg_events.ra, dec=bkg_events.dec)
    
    exp_lon, exp_lat = ra_dec_to_lon_lat(ra=a.bg_data.ra, dec=a.bg_data.dec)
    
    if mask_func is None:
        mask_bkg = np.ones_like(bkg_lat, dtype=bool)
        mask_inj = np.ones_like(inj_lat, dtype=bool)
        mask_exp = np.ones_like(exp_lat, dtype=bool)
    else:
        mask_bkg = mask_func(bkg_events)
        mask_inj = mask_func(inj_events)
        mask_exp = mask_func(a.bg_data)
        
    bkg_events_m = bkg_events[mask_bkg]
    inj_events_m = inj_events[mask_inj]
    exp_events_m = a.bg_data[mask_exp]
        
    n_reps = bin_contributions['n_reps']
    bkg_weights = np.ones_like(bkg_lat[mask_bkg]) / n_reps
    
    plot_bkg = bin_contributions['plot_bkg'][mask_bkg]
    plot_inj = bin_contributions['plot_inj'][mask_inj]
    plot_exp = bin_contributions['plot_exp'][mask_exp]
    bins = bin_contributions['bins']
    xlabel = 'Galactic Latitude $|b|$ / °'
    
    bin_mids = bins[:-1] + 0.5 * np.diff(bins)
    bin_width = np.diff(bins)[0]
    assert np.allclose(np.diff(bins), bin_width)
        
    hist_bkg, _ = np.histogram(plot_bkg, bins=bins, weights=bkg_weights)
    hist_inj, _ = np.histogram(plot_inj, bins=bins)
    hist_exp, _ = np.histogram(plot_exp, bins=bins)
    
    # get contribution to bins
    bin_prob_bkg = bin_contributions['bin_prob_bkg'][mask_bkg]
    bin_prob_inj = bin_contributions['bin_prob_inj'][mask_inj]
    bin_prob_exp = bin_contributions['bin_prob_exp'][mask_exp]
    
    hist_prob_bkg = np.sum(bin_prob_bkg, axis=0) / float(n_reps)
    hist_prob_inj = np.sum(bin_prob_inj, axis=0)
    hist_prob_exp = np.sum(bin_prob_exp, axis=0)
    print('bin_prob_inj', bin_prob_inj.shape)
    print('bin_prob_exp', bin_prob_exp.shape)
    
    
    sig_fraction = np.sum(hist_inj) / np.sum(hist_bkg)
    hist_combined = hist_bkg * (1. - sig_fraction) + hist_inj
    hist_prob_combined = hist_prob_bkg * (1. - sig_fraction) + hist_prob_inj
    
    print('Prob-Hist normalization:', np.sum(hist_prob_exp), np.sum(hist_prob_bkg), np.sum(hist_prob_combined))
    color_cycler = get_color_cycler()
    color_exp = next(color_cycler)
    color_sig = next(color_cycler)
    color_bkg = '0.7'
        
    fig, axes = plt.subplots(3, 1, gridspec_kw={'height_ratios': [2, 1, 1]}, sharex=True, figsize=(9, 6))
    ax = axes[0]
    #ax.hist(plot_bkg, bins=bins, weights=bkg_weights, histtype='step', label='Scrambled background')
    #ax.hist(plot_inj, bins=bins, histtype='step', label='Signal')
    #ax.hist([plot_bkg, plot_inj], weights=[bkg_weights, np.ones_like(plot_inj)], 
    #        bins=bins, histtype='step', label=['Background', 'Signal'], stacked=True)
    #ax.hist(plot_exp, bins=bins, histtype='step', label='Exp')
    #ax.errorbar(bin_mids, hist_exp, yerr=np.sqrt(hist_exp), xerr=bin_width/2., label='Exp Data', fmt='.')
    #ax.errorbar(bin_mids, hist_combined, yerr=np.sqrt(hist_combined), xerr=bin_width/2., label='MC Exp', fmt='.')
    ax.errorbar(bin_mids, hist_prob_combined, yerr=np.sqrt(hist_prob_combined), xerr=bin_width/2., label='MC signal expectation', fmt='.', color=color_sig)
    ax.errorbar(bin_mids, hist_prob_bkg, yerr=np.sqrt(hist_prob_bkg), xerr=bin_width/2., label='Background Scrambles', fmt='.', color=color_bkg)
    ax.errorbar(bin_mids, hist_prob_exp, yerr=np.sqrt(hist_prob_exp), xerr=bin_width/2., label='Exp Data', fmt='.', color=color_exp)
    ax.legend(loc='lower center')
    
    # ----------
    # plot ratio
    # ----------
    axes[1].plot(bins, np.ones_like(bins), color=color_bkg, ls='--')
    axes[1].plot(bin_mids, hist_prob_exp / hist_prob_bkg, color=color_exp, marker='.')
    axes[1].plot(bin_mids, hist_prob_combined / hist_prob_bkg, color=color_sig, marker='.', ls='--')
    
    # ----------
    # plot diff
    # ----------
    axes[2].plot(bins, np.zeros_like(bins), color=color_bkg, ls='--')
    axes[2].plot(bin_mids, hist_prob_exp - hist_prob_bkg, color=color_exp, marker='.')
    axes[2].plot(bin_mids, hist_prob_combined - hist_prob_bkg, color=color_sig, marker='.', ls='--')

    axes[0].set_ylabel('Number of Events')
    axes[1].set_ylabel('Ratio to Bkg')
    axes[2].set_ylabel('Diff to Bkg')
    
    axes[-1].set_xlabel(xlabel)
    axes[-1].set_xlim(bins[0], bins[-1])
    #axes[-1].set_xlim(bins[0], 25)
    #axes[0].set_ylim(5900, 6500)
    return fig, axes

    
def mask_func(df):
    lon, lat = np.rad2deg(ra_dec_to_lon_lat(ra=df.ra, dec=df.dec))
    mask_lon = np.logical_or(
        lon > 300,
        lon < 210,
    )
    mask = np.logical_and(mask_lon, df.energy > 2000)
    #mask = np.logical_and(mask_lon, df.energy < 20000)
    mask = df.energy > -1000
    mask = np.logical_and(mask, df.energy < 50000000)
    mask = np.logical_and(mask, df.sigma < np.deg2rad(150))
    return mask

fig, axes = make_projection_hist(bin_contributions=bin_contributions, mask_func=mask_func)
fig.savefig('{}/latitude_excess.png'.format(plot_dir))

##### Make plots for different energy ranges

In [ ]:
e_ranges = [[0, 1000], [1000, 20000], [20000, float('inf')]]

for e_min, e_max in e_ranges:
    def mask_func(df):
        mask = np.logical_and(df.energy >= e_min, df.energy < e_max)
        return mask

    fig, axes = make_projection_hist(bin_contributions=bin_contributions, mask_func=mask_func)
    axes[0].set_title(r'$E_{\mathrm{reco}} \in $' + '[{:.0f} TeV, {:.0f} TeV]'.format(e_min/1000., e_max/1000.))
    fig.savefig('{}/latitude_excess_E{:.0f}_{:.0f}.png'.format(plot_dir, e_min, e_max))

In [ ]:
ra = np.linspace(0., 2*np.pi, 180)
dec = np.zeros_like(ra)
lon, lat = np.rad2deg(ra_dec_to_lon_lat(ra=ra, dec=dec))

plt.plot(np.rad2deg(ra), lon)
plt.plot(np.rad2deg(ra), lat)
plt.plot(np.rad2deg(ra), np.zeros_like(ra), ls='--', color='0.6')


#### Scratch Space

In [ ]:
import pandas as pd

df = pd.read_hdf('/data/ana/PointSource/DNNCascade/analysis/version-001-p00/MC_NuGen_bfrv1_2153x.hdf', key='df')

In [ ]:
[k for k in df.keys() if 'azi' in k]

In [ ]:
bins = np.linspace(0, 2*np.pi, 30)

fig, ax = plt.subplots(figsize=(9, 6))
ax.hist(df.MCPrimary_azimuth, weights=df.weights, bins=bins, histtype='step', label='MC')
ax.hist(df.azi, weights=df.weights, bins=bins, histtype='step', label='Reco')
ax.hist(a.bg_data.azimuth, bins=bins, histtype='step', label='Exp')

ax.legend()